In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import math
import seaborn as sns
from matplotlib import cm
import sys
import random

import itertools
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve,auc
%matplotlib inline




In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    #credits - sklearn
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    if normalize:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    else:    
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(int(cm[i, j]), fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



In [ ]:
class1=np.loadtxt('./data_set/syn/class1.txt')
class2=np.loadtxt('./data_set/syn/class2.txt')
np.random.shuffle(class1)
np.random.shuffle(class2)

In [ ]:
class Gaussian:
    
    def __init__(self, mu, sigma):
        
        self.mu = mu
        self.sigma = sigma
        #print mu,sigma
    def logpdf(self,datum):
        size = len(datum)
        det = np.linalg.det(self.sigma)

        N =  ((2*math.pi)**size * det)**0.5 
        x_mu = (datum - self.mu)
        inv = np.linalg.inv(self.sigma)        
        result = -0.5 * np.matmul(x_mu , np.matmul(inv, np.transpose(x_mu)))
        return  result-np.log(N)
    
    
    def pdf(self, datum):
        size = len(datum)
        det = np.linalg.det(self.sigma)

        N =  ((2*math.pi)**size * det)**0.5 
        x_mu = (datum - self.mu)
        inv = np.linalg.inv(self.sigma)        
        result = math.pow(math.e, -0.5 * np.matmul(x_mu , np.matmul(inv, np.transpose(x_mu))))
        return  result/N
    
    def __repr__(self):
        return 'Gaussian({0:4.6}, {1})'.format(self.mu, self.sigma)
    def plot(self):
        mu=self.mu
        sig=self.sigma
        
        N = 60
        X = np.linspace(mu[0]-3*sig[0][0]**0.5, mu[0]+3*sig[0][0]**0.5, N)
        Y = np.linspace(mu[1]-3*sig[1][1]**0.5, mu[1]+3*sig[1][1]**0.5, N)

        X, Y = np.meshgrid(X, Y)

        zs = np.array([self.pdf([x,y]) for x,y in zip(np.ravel(X), np.ravel(Y))])
        Z = zs.reshape(X.shape)

        #ax.plot_surface(X, Y, Z,alpha=0.5, rstride=3, cstride=3, linewidth=1, antialiased=True,cmap=cm.inferno)

        cset = ax.contour(X, Y, Z)#,cmap=cm.inferno)

        

In [ ]:
N=class1.shape[0]

c1_train=class1[:int(math.ceil(N*7/10))]
c1_valid=class1[int(math.ceil(N*7/10)):int(math.ceil(N*9/10))]
c1_test=class1[int(math.ceil(N*9/10)):]
N=class2.shape[0]
c2_train=class2[:int(math.ceil(N*7/10))]
c2_valid=class2[int(math.ceil(N*7/10)):int(math.ceil(N*9/10))]
c2_test=class2[int(math.ceil(N*9/10)):]


In [ ]:
c1_mu=np.mean(c1_train,axis=0)
temp=np.vstack((c1_train[:,0],c1_train[:,1]))
c1_sig=np.cov(temp)

print c1_train.T,temp

c2_mu=np.mean(c2_train,axis=0)
temp=np.vstack((c2_train[:,0],c2_train[:,1]))
c2_sig=np.cov(temp)

gas_c1 = Gaussian(c1_mu, c1_sig)
gas_c2 = Gaussian(c2_mu, c2_sig)
print gas_c1

In [ ]:
ax = plt.gca()
ax = sns.kdeplot(c1_train[:,0],c1_train[:,1],cmap="Reds", shade=True, shade_lowest=False)
ax = sns.kdeplot(c2_train[:,0],c2_train[:,1],cmap="Blues", shade=True, shade_lowest=False)

plt.savefig('./results/sin',format='eps')
plt.show()

from sklearn import mixture
 
def fit_samples(samples):
    gmix = mixture.GMM(n_components=2, covariance_type='full')
    gmix.fit(samples)
    #print gmix.means_
    colors=[]
    for i in gmix.predict(samples):
        if i==0:
            colors+='r'
        elif i==1:
            colors+='g'
        
    ax = plt.gca()
    ax.scatter(samples[:,0], samples[:,1], c=colors, alpha=0.8)
    plt.show()
fit_samples(class1+class2)

In [ ]:
class GaussianMixture:
    
    def __init__(self, data,n):
        self.data = data
        self.N=n
        kmeans = KMeans(n_clusters=n,random_state=0).fit(data)
        
        mu=[]
        for i in range(n):
            mu.append(kmeans.cluster_centers_[i])
        
        #print mu
        c=[]
        for k in range(n):
            c.append(np.array([i for i in data if kmeans.predict([i])[0]==k]))
        
        
        
        
        
        sig=[]
        
        for i in range(n):
            temp=np.vstack((c[i][:,0],c[i][:,1]))
            sig.append(np.cov(temp))
            
        
        
        self.gas=[]
        for i in range(n):
            self.gas.append(Gaussian(mu[i],sig[i]))
        
        
        self.w = np.full(n,1.0/n)
        
        self.logN=0
    def E(self):
        v=[]
        self.logN=0
        gamma=[]
        for i in self.data:
            gamma=[]
            for k in range(self.N):
                gamma.append(self.gas[k].pdf(i)*self.w[k])
            
            
            total_gamma = sum(gamma)
            
            
            
            for k in range(self.N):
                gamma[k]/=total_gamma
            
            
            for k in range(self.N):
                
                self.logN += (self.gas[k].logpdf(i))+np.log(self.w[k])
            
            #sys.exit()    
            
            v.append(gamma)
            
        return v

    def M(self,gammas):
        Nk=[]
        for i in range(self.N):
            Nk.append(sum(gammas[:,i]))
            
        
        
        for k in range(self.N):
            self.gas[k].mu=np.sum(i*j for (i,j) in zip(gammas[:,k], self.data))/Nk[k]
        
        for k in range(self.N):
            self.gas[k].sigma=np.sum([(i* (np.outer((j - self.gas[k].mu),np.array(j - self.gas[k].mu)))) for (i,j) in zip(gammas[:,k], self.data)],axis=0)/Nk[k]
        
        for k in range(self.N):
            
            self.w[k]=Nk[k]/len(self.data)
            
        

    def __str__(self):
        return 'Logliklywd :'+str(self.logN)
    
    def pdf(self, x):
        val=0
        for i in range(self.N):
            val+=(self.w[i])*self.gas[i].pdf(x)
        return val
    def likelihood(self,x):
        val=0
        for i in range(self.N):
            val+=self.gas[i].pdf(x)*self.w[i]
        return val
    def prefict(self,x):
        
        max=0
        cls=-1
        for i in range(self.N):
            if(self.gas[i].pdf(x)>max):
                max=self.gas[i].pdf(x)
                cls=i
        return cls                                                


In [ ]:
models=[]
accs=[]
for clusters in range(2,3):
    iters = 5
    #clusters=1
    train_data=np.vstack([c1_train,c2_train])
    test_data=np.vstack([c1_valid,c2_valid])
    
    oldN=0
    mix1 = GaussianMixture(c1_train,clusters)#c2_train,4)
    for i in range(iters):
        v=mix1.E()
        v=np.array(v)
        mix1.M(v)
        if mix1.logN==oldN:
            break
        oldN=mix1.logN
        
        #print 1,mix1.logN
    
    oldN=0
    mix2 = GaussianMixture(c2_train,clusters)#c2_train,4)
    for i in range(iters):
        v=mix2.E()
        v=np.array(v)
        mix2.M(v)
        if mix2.logN==oldN:
            break
        oldN=mix2.logN
        
        #print 2,mix2.logN



    total=0
    crt=0
    wrg=0
    tx=0
    colors=[]
    samples=c1_test#+c2_valid
    for s in samples:
        total+=1
        tmp=[mix1.likelihood(s),mix2.likelihood(s)]
        i=tmp.index(max(tmp))
        if i==0:
            tx+=1
            colors+='b'
        elif i==1:
            colors+='g'
    crt=tx
    wrg=total-crt

    total_crt=crt
    total_wrg=wrg


    total=0
    crt=0
    wrg=0
    tx=0
    samples=c2_test#+c2_valid
    for s in samples:
        total+=1
        tmp=[mix1.likelihood(s),mix2.likelihood(s)]
        i=tmp.index(max(tmp))
        if i==1:
            tx+=1
            colors+='b'
        elif i==0:
            colors+='g'
    crt=tx
    wrg=total-crt

    total_crt+=crt
    total_wrg+=wrg
    a=total_crt*1.0/(total_crt+total_wrg)
    print 'c = ',clusters*2,' accuracy = ',a
    accs.append([clusters*2,a])
    

    ax = plt.gca()
    #ax.scatter(samples[:,0], samples[:,1], c=colors, alpha=0.8)

    ax = sns.kdeplot(c1_test[:,0],c1_test[:,1],cmap="Reds", shade=True, shade_lowest=False)
    ax = sns.kdeplot(c2_test[:,0],c2_test[:,1],cmap="Blues", shade=True, shade_lowest=False)



    for i in range(clusters):
        mix1.gas[i].plot()
        #x,y=np.random.multivariate_normal(mix1.gas[i].mu,mix1.gas[i].sigma,500).T
        #ax=sns.kdeplot(x, y, n_levels=5)
    for i in range(clusters):
        mix2.gas[i].plot()
    plt.savefig('./results/sin_full_'+str(clusters*2),format='eps')
    plt.show()
    models.append((mix1,mix2))
    if a==1.0:
        break
ax = plt.gca()
accs=np.array(accs)

ax.plot(accs[:,0],accs[:,1])
plt.savefig('./results/sin_full',format='eps')
    
plt.show()

In [ ]:
#conf matrix
cl=2
all_predictions=[]
for mix1,mix2 in models:
    cf=np.zeros((2,2))
    count=0
    #print 'testing class1'
    predictions=[]
    for s in c1_test:

        tmp=[mix1.likelihood(s),mix2.likelihood(s)]
        i=tmp.index(max(tmp))
        predictions.append([0,i])
        if i==0:
            cf[0,0]+=1


        elif i==1:
            cf[0,1]+=1

    for s in c2_test:

        tmp=[mix1.likelihood(s),mix2.likelihood(s)]
        i=tmp.index(max(tmp))
        predictions.append([1,i])
        if i==0:
            cf[1,0]+=1

        elif i==1:
            cf[1,1]+=1

    all_predictions.append(predictions)
    plot_confusion_matrix(cf,['class_1','class_2'])
    
    plt.savefig('./results/syn_full_conf_k='+str(cl),format='eps')
    plt.show()
    plot_confusion_matrix(cf,['class_1','class_2'],normalize=True)
    plt.savefig('./results/syn_full_conf_nrmlized_k='+str(cl),format='eps')
    cl+=2
    plt.show()


In [ ]:
plt.figure()
i=2
for predictions in all_predictions:
    predictions=np.array(predictions)

    fpr, tpr, _ = roc_curve(predictions[:,0], predictions[:,1])
    roc_auc = auc(fpr, tpr)
    #print fpr,tpr
    lw = 2
    plt.plot(fpr, tpr, 
             lw=lw, label='k = %d (area = %0.2f)'%(i,roc_auc))
    i=i+2
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
plt.savefig('./results/syn_roc',format='eps')
plt.show()